In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import *
import xgboost as xgb
import lightgbm as lgb
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.cross_validation import train_test_split
from sklearn.metrics import *
from sklearn.model_selection import StratifiedKFold
import random

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
print(train.shape)
train.head()
test.head()

In [ ]:
# found that 'ps_calc_15_bin','ps_calc_16_bin','ps_calc_17_bin','ps_calc_18_bin','ps_calc_19_bin','ps_calc_20_bin' are almost uniform distribution on target = 1
train.drop(['ps_calc_15_bin','ps_calc_16_bin','ps_calc_17_bin','ps_calc_18_bin','ps_calc_19_bin','ps_calc_20_bin'],axis=1,inplace=True)
# found that 'ps_calc_01','ps_calc_02','ps_calc_03' are uniform distribution on target = 1
train.drop(['ps_calc_01','ps_calc_02','ps_calc_03'],axis=1,inplace=True)
# found that 'ps_calc_04','ps_calc_09' are uniform distribution on target = 1
train.drop(['ps_calc_04','ps_calc_09'],axis=1,inplace=True)
# the only sample whose 'ps_car_12' value is -1 and the 'target' value is 0, I think this is a noise sample
train.drop(298018,axis=0,inplace=True) 

# 'ps_reg_01','ps_reg_02','ps_reg_03' are correlated and their combination's distribution looks great, like a normal distribution
ps_reg = train['ps_reg_01'].add(train['ps_reg_02']).add(train['ps_reg_03'])
ps_reg.name = 'ps_reg'
train = pd.concat([train,ps_reg],axis=1)
train.drop(['ps_reg_01','ps_reg_02','ps_reg_03'],axis=1,inplace=True)
#train.drop(['ps_reg_01','ps_reg_02'],axis=1,inplace=True)
'''
# 'ps_car_12','ps_car_13' are correlated and their combination's distribution looks great
ps_car = train['ps_car_12'].add(train['ps_car_13'])#.add(train['ps_car_14'])
ps_car.name = 'ps_car_1213'
train = pd.concat([train,ps_car],axis=1)
#train.drop(['ps_car_12','ps_car_13','ps_car_14'],axis=1,inplace=True)
train.drop(['ps_car_12','ps_car_13'],axis=1,inplace=True)
'''
target = train['target']
train.drop('target',axis=1,inplace=True)
train.drop('id',axis=1,inplace=True)

In [ ]:

cat_features = []
bin_features = []
continue_features = []
ordinal_features = []
for i in train.columns[2:]:
    if i.endswith('_bin'):
        bin_features.append(i)
    elif i.endswith('_cat'):
        cat_features.append(i)
    elif train[i].dtype == 'float64':
        continue_features.append(i)
    elif train[i].dtype == 'int64':
        ordinal_features.append(i)
print ('categorical features:')
print (cat_features)
print ('binary features:')
print (bin_features)
print ('continue features:')
print (continue_features)
print ('ordinal features:')
print (ordinal_features)

In [ ]:
for feature in cat_features:
    dummies = pd.get_dummies(train[feature],prefix=feature)[1:]
    train = pd.concat([train,dummies],axis=1)
    train.drop(feature,axis=1,inplace=True)

In [ ]:
# found that 'ps_calc_15_bin','ps_calc_16_bin','ps_calc_17_bin','ps_calc_18_bin','ps_calc_19_bin','ps_calc_20_bin' are almost uniform distribution on target = 1
test.drop(['ps_calc_15_bin','ps_calc_16_bin','ps_calc_17_bin','ps_calc_18_bin','ps_calc_19_bin','ps_calc_20_bin'],axis=1,inplace=True)
# found that 'ps_calc_01','ps_calc_02','ps_calc_03' are uniform distribution on target = 1
test.drop(['ps_calc_01','ps_calc_02','ps_calc_03'],axis=1,inplace=True)
# found that 'ps_calc_04','ps_calc_09' are uniform distribution on target = 1
test.drop(['ps_calc_04','ps_calc_09'],axis=1,inplace=True)
# the only sample whose 'ps_car_12' value is -1 and the 'target' value is 0, I think this is a noise sample
test.drop(298018,axis=0,inplace=True)

# 'ps_reg_01','ps_reg_02','ps_reg_03' are correlated and their combination's distribution looks great, like a normal distribution
ps_reg = test['ps_reg_01'].add(test['ps_reg_02']).add(test['ps_reg_03'])
ps_reg.name = 'ps_reg'
test = pd.concat([test,ps_reg],axis=1)
test.drop(['ps_reg_01','ps_reg_02','ps_reg_03'],axis=1,inplace=True)
#test.drop(['ps_reg_01','ps_reg_02'],axis=1,inplace=True)
'''
# 'ps_car_12','ps_car_13' are correlated and their combination's distribution looks great
ps_car = test['ps_car_12'].add(test['ps_car_13'])#.add(test['ps_car_14'])
ps_car.name = 'ps_car_1213'
test = pd.concat([test,ps_car],axis=1)
#test.drop(['ps_car_12','ps_car_13','ps_car_14'],axis=1,inplace=True)
test.drop(['ps_car_12','ps_car_13'],axis=1,inplace=True)
'''
#target = test['target']
#test.drop('target',axis=1,inplace=True)
test.drop('id',axis=1,inplace=True)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(train, target, test_size = 0.25, random_state = 0)
'''
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
'''

In [ ]:
def gini(y, pred):
    fpr, tpr, thr = metrics.roc_curve(y, pred, pos_label=1)
    g = 2 * metrics.auc(fpr, tpr) - 1
    return g

def gini_normalized(y,pred):
    return  gini(y, pred) / gini(y, y)
def cv_score(model,X,Y,cv=5):
    kf = StratifiedKFold(n_splits=cv)
    X = X.as_matrix()
    Y= Y.as_matrix()
    score = np.zeros(4)
    for train_index,test_index in kf.split(X,Y):
        train_x, test_x = X[train_index],X[test_index]
        train_y, test_y = Y[train_index],Y[test_index]
        model.fit(train_x,train_y)
        pre = model.predict(test_x)
        pre_proba = model.predict_proba(test_x).T[1]
        temp = [metrics.accuracy_score(pre,test_y),
                metrics.fbeta_score(test_y,pre,beta=2.0),
                metrics.roc_auc_score(test_y,pre_proba),
                gini_normalized(test_y,pre_proba)]
        score = np.mean([score,temp],axis=0)
    return score
def saveFigure(x,scores,x_label):
    scores = np.matrix(scores)
    fig = plt.figure(figsize = (12,10))
    length = len(np.ravel(scores[0]))
    data = pd.DataFrame(data=scores,columns=['acc','F2 score','ROC_AUC','Gini'],index=x)
    plt.plot(data)
    plt.xlabel(x_label,fontsize=18)
    plt.ylabel('CV-scores',fontsize=18)
    plt.legend(['acc','F2 score','ROC_AUC','Gini'])
    #plt.show()
    fig.savefig(x_label+'.png', dpi=fig.dpi)

In [ ]:
estimators = np.linspace(50,350,7,dtype=int)
est_scores = []
for est in estimators:
    clf = xgb.XGBClassifier(n_estimators = est, max_depth = 5, silent = True, n_jobs = -1,
                        booster='gbtree',random_state=7, subsample = 0.8, colsample_bytree = 0.8,
                        learning_rate=0.01, objective = 'binary:logistic')#scale_pos_weight
    est_scores.append(cv_score(clf,x_train,y_train))
est_scores = np.array(est_scores)

np.savetxt('xgb_estimators.txt', est_scores, delimiter=',')
saveFigure(estimators, est_scores, 'Estimators')

In [ ]:
np.loadtxt(fname='xgb_iters.txt',delimiter=',')

In [ ]:
np.matrix(iter_scores).T

In [ ]:
saveFigure(iterations,iter_scores,'estimators')

# Random Forest

# n_estimators (1~300)

In [ ]:
N_forest = 300
rf = ensemble.RandomForestClassifier(n_estimators=N_forest, max_depth=5, 
                                     min_samples_leaf=4, n_jobs = -1, oob_score =True, 
                                     max_features='auto', random_state=0, class_weight='balanced_subsample')
rf.fit(x_train, y_train)
#features = train.columns.values
print("----- Training Done -----")

In [ ]:
votes = rf.estimators_
vote_results = []
for vote in votes:
    vote_results.append(vote.predict_proba(x_test).T[1])
mat = np.array(vote_results)
result_acc = []
result_fb = []
result_auc = []
res = np.mean(mat[:10],axis=0)
for i in range(10,N_forest):
    res = (res + mat[i] ) / 2.0
    temp = res[:]
    temp[temp >= 0.5] = 1
    temp[temp < 0.5] = 0
    result_acc.append(accuracy_score(y_test,temp))
    result_fb.append(fbeta_score(y_test,temp,beta=2))
    result_auc.append(roc_auc_score(y_test,res))

In [ ]:
plt.plot(result_acc) # n_estimators=200

In [ ]:
plt.plot(result_fb) # n_estimators=200

In [ ]:
plt.plot(result_auc) # n_estimators=200

## max_depth = (1~10), min_samples_leaf(1~10)

In [ ]:
depths = np.linspace(2,10,9)
result_acc = []
result_fb = []
result_auc = []
for depth in depths:
    rf = ensemble.RandomForestClassifier(n_estimators=100, max_depth=depth, 
                                         min_samples_leaf=4, n_jobs = -1, oob_score =True, 
                                         max_features='auto', random_state=0, class_weight='balanced_subsample')
    rf.fit(x_train, y_train)
    temp = rf.predict(x_test)
    temp_proba = rf.predict_proba(x_test).T[1]
    result_acc.append(accuracy_score(y_test,temp))
    result_fb.append(fbeta_score(y_test,temp,beta=2))
    result_auc.append(roc_auc_score(y_test,temp_proba))

In [ ]:
plt.plot(result_acc)

In [ ]:
plt.plot(result_fb)

In [ ]:
plt.plot(result_auc)

In [ ]:
leafs = np.linspace(2,10,9)
result_acc = []
result_fb = []
result_auc = []
for leaf in leafs:
    rf = ensemble.RandomForestClassifier(n_estimators=200, max_depth=5, 
                                         min_samples_leaf = int(leaf), n_jobs = -1, oob_score =True, 
                                         max_features='auto', random_state=0, class_weight='balanced_subsample')
    rf.fit(x_train, y_train)
    temp = rf.predict(x_test)
    temp_proba = rf.predict_proba(x_test).T[1]
    result_acc.append(accuracy_score(y_test,temp))
    result_fb.append(fbeta_score(y_test,temp,beta=2))
    result_auc.append(roc_auc_score(y_test,temp_proba))

In [ ]:
plt.plot(result_acc)

In [ ]:
plt.plot(result_fb)

In [ ]:
plt.plot(result_auc)

In [ ]:
def gini_rf(pred, y):
    #y = y.get_label()
    return 'gini', gini(y, pred) / gini(y, y)

In [ ]:
# After hyperparameter selection, the good hyperparameters are n_estimators=200, max_depth=5, min_samples_leaf = 4
rf = ensemble.RandomForestClassifier(n_estimators=200, max_depth=5, 
                                         min_samples_leaf = 4, n_jobs = -1, oob_score =True, 
                                         max_features='auto', random_state=0, class_weight='balanced_subsample')
rf.fit(x_train, y_train)
temp = rf.predict(x_test)
temp_proba = rf.predict_proba(x_test).T[1]

print (gini_rf(temp_proba,y_test))
print ("Random Forest")
print ("accuracy",accuracy_score(y_test,temp))
print ("F2 score", fbeta_score(y_test,temp,beta=2))
print ("ROC_AUC:",roc_auc_score(y_test, temp_proba))

In [ ]:
importance = rf.feature_importances_
features = train.columns.values
plt.figure(figsize=(10,12))
importance = pd.DataFrame(data=importance,index=features,columns=['importance'])
importance.sort_values('importance', ascending=False, inplace=True)
# Bar plot
# Order the bars descending on target mean
sns.barplot(x='importance', y=importance.index, data=importance)
sns.despine(left=True, bottom=True)

# XGBoost

In [ ]:
def gini(y, pred):
    fpr, tpr, thr = metrics.roc_curve(y, pred, pos_label=1)
    g = 2 * metrics.auc(fpr, tpr) - 1
    return g

def gini_xgb(pred, y):
    y = y.get_label()
    return 'gini', gini(y, pred) / gini(y, y)
random.seed( 3 )
x1, x2, y1, y2 = model_selection.train_test_split(train, target, test_size=0.3, random_state=6)
watchlist = [(xgb.DMatrix(x1, y1), 'train'), (xgb.DMatrix(x2, y2), 'valid')]
params = {'eta': 0.09, 'max_depth': 6, 'objective': 'binary:logistic', 'seed': 16, 'silent': True, 'colsample_bytree': 0.6}
model = xgb.train(params, xgb.DMatrix(x1, y1), 1000,  watchlist, feval=gini_xgb, maximize=True, verbose_eval=50, early_stopping_rounds=100)

In [ ]:
xgb.plot_importance(model,max_num_features =20)

In [ ]:
sns.jointplot(x="ps_car_12", y="ps_car_13", data=train);
sns.distplot(ps_car);

In [ ]:
sns.pairplot(train[['ps_reg_01','ps_reg_02','ps_reg_03']]);
ps_reg = train['ps_reg_01'].add(train['ps_reg_02']).add(train['ps_reg_03'])
sns.distplot(ps_reg);

# LGBM

In [ ]:
# coding: utf-8
# pylint: disable = invalid-name, C0111
import json
try:
    import cPickle as pickle
except:
    import pickle

# load or create your dataset
print('Load data...')
num_train, num_feature = x_train.shape

# create dataset for lightgbm
# if you want to re-use data, remember to set free_raw_data=False
lgb_train = lgb.Dataset(x_train, y_train, free_raw_data=False) # weight=W_train,
lgb_eval = lgb.Dataset(x_test, y_test, reference=lgb_train, free_raw_data=False) #weight=W_test,

columns = train.columns.values

# specify your configurations as a dict
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc,binary_logloss',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'metric_freq' : 10,
    'verbose': 0,
    'is_unbalance': True
}

# generate a feature name
feature_name = ['feature_' + columns[col] for col in range(num_feature)]

print('Start training...')
# feature_name and categorical_feature
gbm = lgb.train(params,
                lgb_train,
                #num_boost_round=10,
                valid_sets=lgb_train,  # eval training data
                feature_name=feature_name) # ,                categorical_feature=[21]

# check feature name
print('Finish first 10 rounds...')
print('7th feature name is:', repr(lgb_train.feature_name[6]))

# save model to file
gbm.save_model('model.txt')

# dump model to json (and save to file)
print('Dump model to JSON...')
model_json = gbm.dump_model()

with open('model.json', 'w+') as f:
    json.dump(model_json, f, indent=4)

# feature names
print('Feature names:', gbm.feature_name())

# feature importances
print('Feature importances:', list(gbm.feature_importance()))


In [ ]:
# load model to predict
print('Load model to predict')
bst = lgb.Booster(model_file='model.txt')
# can only predict with the best iteration (or the saving iteration)
y_pred = bst.predict(x_test)
print (y_test)
print (y_pred)
temp = y_pred[:]
temp[temp >= 0.5] = 1
temp[temp < 0.5] = 0
print ("accuracy",accuracy_score(y_test,temp))
print ("F2 score", fbeta_score(y_test,temp,beta=2))
print ("ROC_AUC:",roc_auc_score(y_test, temp_proba))

In [ ]:

# load model to predict
print('Load model to predict')
bst = lgb.Booster(model_file='model.txt')
# can only predict with the best iteration (or the saving iteration)
y_pred = bst.predict(X_test)
# eval with loaded model
print('The rmse of loaded model\'s prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)

# dump model with pickle
with open('model.pkl', 'wb') as fout:
    pickle.dump(gbm, fout)
# load model with pickle to predict
with open('model.pkl', 'rb') as fin:
    pkl_bst = pickle.load(fin)
# can predict with any iteration when loaded in pickle way
y_pred = pkl_bst.predict(X_test, num_iteration=7)
# eval with loaded model
print('The rmse of pickled model\'s prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)

# continue training
# init_model accepts:
# 1. model file name
# 2. Booster()
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=10,
                init_model='model.txt',
                valid_sets=lgb_eval)

print('Finish 10 - 20 rounds with model file...')

# decay learning rates
# learning_rates accepts:
# 1. list/tuple with length = num_boost_round
# 2. function(curr_iter)
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=10,
                init_model=gbm,
                learning_rates=lambda iter: 0.05 * (0.99 ** iter),
                valid_sets=lgb_eval)

print('Finish 20 - 30 rounds with decay learning rates...')

# change other parameters during training
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=10,
                init_model=gbm,
                valid_sets=lgb_eval,
                callbacks=[lgb.reset_parameter(bagging_fraction=[0.7] * 5 + [0.6] * 5)])

print('Finish 30 - 40 rounds with changing bagging_fraction...')


# self-defined objective function
# f(preds: array, train_data: Dataset) -> grad: array, hess: array
# log likelihood loss
def loglikelood(preds, train_data):
    labels = train_data.get_label()
    preds = 1. / (1. + np.exp(-preds))
    grad = preds - labels
    hess = preds * (1. - preds)
    return grad, hess


# self-defined eval metric
# f(preds: array, train_data: Dataset) -> name: string, value: array, is_higher_better: bool
# binary error
def binary_error(preds, train_data):
    labels = train_data.get_label()
    return 'error', np.mean(labels != (preds > 0.5)), False


gbm = lgb.train(params,
                lgb_train,
                num_boost_round=10,
                init_model=gbm,
                fobj=loglikelood,
                feval=binary_error,
                valid_sets=lgb_eval)

print('Finish 40 - 50 rounds with self-defined objective function and eval metric...')

print('Start a new training job...')


# callback
def reset_metrics():
    def callback(env):
        lgb_eval_new = lgb.Dataset(X_test, y_test, reference=lgb_train)
        if env.iteration - env.begin_iteration == 5:
            print('Add a new valid dataset at iteration 5...')
            env.model.add_valid(lgb_eval_new, 'new valid')
    callback.before_iteration = True
    callback.order = 0
    return callback


gbm = lgb.train(params,
                lgb_train,
                num_boost_round=10,
                valid_sets=lgb_train,
                callbacks=[reset_metrics()])

print('Finish first 10 rounds with callback function...')